In [1]:
import mlflow
import pandas as pd

# from startup import setup_environment
from src.config import MLFLOW_TRACKING_URI

In [ ]:


def get_best_models_per_experiment(tracking_uri=None):
    """
    For each experiment, find the run with smallest RMSE, smallest MAE,
    and smallest absolute ME. Return a DataFrame with one row per such run.
    """
    if tracking_uri:
        mlflow.set_tracking_uri(tracking_uri)

    results = []
    experiments = mlflow.search_experiments()

    for exp in experiments:
        runs_df = mlflow.search_runs(experiment_ids=[exp.experiment_id])
        if runs_df.empty:
            continue

        # Prepare a DataFrame with only the columns we need
        needed = {
            'run_id': runs_df['run_id'],
            'run_name': runs_df.get('tags.mlflow.runName', runs_df['run_id']),
            'dataset_name': runs_df.get('params.dataset_name', None),
            'model': runs_df.get('tags.model', None),
            'model_family': runs_df.get('tags.model_family', None),
            'loss': runs_df.get('tags.loss', None),
            'RMSE': runs_df.get('metrics.rmse', float('nan')),
            'MAE': runs_df.get('metrics.mae', float('nan')),
            'ME': runs_df.get('metrics.me', float('nan')),
            'R2': runs_df.get('metrics.r2', float('nan'))
        }
        df = pd.DataFrame(needed)

        # Drop rows missing critical metrics
        df = df.dropna(subset=['RMSE', 'MAE', 'ME'])

        if df.empty:
            continue

        # Add absolute ME for selection
        df['abs_ME'] = df['ME'].abs()

        # Find best runs
        best_rmse_idx = df['RMSE'].idxmin()
        best_mae_idx = df['MAE'].idxmin()
        best_me_idx = df['abs_ME'].idxmin()

        # Collect results (avoid duplicates if same run is best for multiple metrics)
        seen = set()
        for idx in [best_rmse_idx, best_mae_idx, best_me_idx]:
            if idx in seen:
                continue
            seen.add(idx)
            row = df.loc[idx]
            results.append({
                'experiment_name': exp.name,
                'run_name': row['run_name'],
                'run_id': row['run_id'],
                'dataset_name': row['dataset_name'],
                'model': row['model'],
                'model_family': row['model_family'],
                'loss': row['loss'],
                'RMSE': row['RMSE'],
                'MAE': row['MAE'],
                'ME': row['ME'],
                'R2': row['R2']
            })

    return pd.DataFrame(results)

In [3]:
best_models = get_best_models_per_experiment(MLFLOW_TRACKING_URI)
best_models

,experiment_name,run_name,run_id,dataset_name,model,model_family,loss,RMSE,MAE,ME,R2
0,ElasticNet_hourly,ElasticNet_alpha0.1_l10.1,b6c03938214a428cb69eb112bf55030a,dataset_v5_full_hourly,ElasticNet,linear,None,24.267768,17.722027,4.089057,0.773533
1,ElasticNet_hourly,ElasticNet_alpha0.0001_l10.5,1e1b8101d96b49878457c4d4de7eab61,dataset_v5_full_hourly,ElasticNet,linear,None,24.350099,17.671575,3.394829,0.771994
2,ElasticNet_hourly,ElasticNet_alpha0.0001_l11.0,740f0dad5850401ea682ed42db87da58,dataset_v5_slim_hourly,ElasticNet,linear,None,24.559297,17.901805,0.590192,0.768059
3,Ridge_hourly,Ridge_alpha1000,29367968e9e549a49bf0ab74ed85078b,dataset_v5_full_hourly_selected_rfecv,Ridge,linear,None,24.398227,17.905102,2.678738,0.771092
4,Ridge_hourly,Ridge_alpha0.0001,beebac1c0a004005bedf88a676eac0da,dataset_v5_full_hourly_selected_rfecv,Ridge,linear,None,24.441195,17.741454,1.360741,0.770285
5,Lasso_hourly,Lasso_alpha0.01,653cb87f855b42cc8ba0674fe75daa5a,dataset_v5_full_hourly,Lasso,linear,None,24.369862,17.707240,3.827181,0.771624
6,Lasso_hourly,Lasso_alpha0.0001,99f26b8eade4481b9252038b8ba1ea09,dataset_v5_slim_hourly,Lasso,linear,None,24.559297,17.901805,0.590192,0.768059
7,XGBoost_hourly,XGBoost_hourly_A_mae,23c6241490984d93ae04752ee61571ac,dataset_v5_full_hourly,XGBoost,boosted_tree,mae,20.132833,11.973198,1.292681,0.844133
8,XGBoost_hourly,XGBoost_rand_trial_13,ecbb6e460ff3430fb9bb5411ebd09200,dataset_v5_slim_hourly,XGBoost,boosted_tree,mae,20.492554,11.951262,-0.983998,0.838513
9,XGBoost_hourly,XGBoost_hourly_B_huber,fc43d42dd7ea45cd827823ade033d091,dataset_v5_slim_hourly,XGBoost,boosted_tree,huber,22.250265,13.070874,0.042017,0.809623


In [21]:
def select_top_per_group(best_runs_df):
    conditions = [
        (best_runs_df['model_family'].str.lower() == 'linear'),
        (best_runs_df['model'].str.lower().str.contains('xgboost', na=False)),
        (best_runs_df['model'].str.lower().str.contains('lgbm|lightgbm', na=False, regex=True)),
        (best_runs_df['model'].str.lower().str.contains('catboost', na=False))
    ]
    group_names = ['linear', 'XGBoost', 'LGBM', 'CatBoost']
    best_runs_df['group'] = None
    for cond, name in zip(conditions, group_names):
        best_runs_df.loc[cond, 'group'] = name

    grouped = best_runs_df.dropna(subset=['group']).copy()
    if grouped.empty:
        return pd.DataFrame()

    # Compute overall ranks (across all runs that belong to a group)
    grouped['rank_rmse'] = grouped['RMSE'].rank(ascending=True, method='min')
    grouped['rank_mae'] = grouped['MAE'].rank(ascending=True, method='min')
    grouped['rank_me'] = grouped['ME'].abs().rank(ascending=True, method='min')
    grouped['avg_rank'] = grouped[['rank_rmse', 'rank_mae', 'rank_me']].mean(axis=1)

    selected = []
    for grp in grouped['group'].unique():
        sub = grouped[grouped['group'] == grp]
        # Take top 2 based on avg_rank (lower is better)
        top2 = sub.nsmallest(2, 'avg_rank')
        selected.append(top2)

    final_df = pd.concat(selected, ignore_index=True)
    return final_df

In [24]:
best_models_by_group = select_top_per_group(best_models)
best_models_by_group

,experiment_name,run_name,run_id,dataset_name,model,model_family,loss,RMSE,MAE,ME,R2,group,rank_rmse,rank_mae,rank_me,avg_rank
0,linear,lasso_alpha_0.1_none,d091f1c8342b4c649f86931157f07617,dataset_v3,None,linear,None,24.756247,14.352092,0.058932,0.774722,linear,19.0,11.0,5.0,11.666667
1,linear,lasso_alpha_{alpha}_{ttransform},d8deb4fefd374fbfbd0fe86ab0c61fc9,dataset_v3,None,linear,None,26.233866,14.916231,0.043523,0.747028,linear,21.0,12.0,4.0,12.333333
2,XGBoost_hourly,XGBoost_rand_trial_13,ecbb6e460ff3430fb9bb5411ebd09200,dataset_v5_slim_hourly,XGBoost,boosted_tree,mae,20.492554,11.951262,-0.983998,0.838513,XGBoost,4.0,2.0,13.0,6.333333
3,XGBoost_hourly,XGBoost_hourly_B_huber,fc43d42dd7ea45cd827823ade033d091,dataset_v5_slim_hourly,XGBoost,boosted_tree,huber,22.250265,13.070874,0.042017,0.809623,XGBoost,8.0,8.0,3.0,6.333333
4,CatBoost_hourly,CatBoost_hourly_A_mae,daa9083db9664b42979f1793543e642f,dataset_v5_full_hourly_selected_rfecv,CatBoost,boosted_tree,mae,21.037518,12.397467,-0.114859,0.829810,CatBoost,7.0,5.0,6.0,6.000000
5,CatBoost_hourly,CatBoost_hourly_A_mae,cdde162175474a468a8894876a96ca61,dataset_v5_full_hourly_selected_shap,CatBoost,boosted_tree,mae,20.560067,12.451075,0.659319,0.837447,CatBoost,6.0,6.0,11.0,7.666667
6,LGBM_hourly,LGBM_hourly_A_mae,31b03ec78a17495a8f00e19c03a3335b,dataset_v5_full_hourly_selected_rfecv,LGBM,boosted_tree,mae,20.523124,12.041540,0.004895,0.838031,LGBM,5.0,4.0,1.0,3.333333
7,LGBM_hourly,LGBM_hourly_A_mae,b1dfd4b6ba58496fa6c3449b59de21dc,dataset_v5_full_hourly_selected_shap,LGBM,boosted_tree,mae,20.063736,11.868258,-0.640587,0.845201,LGBM,2.0,1.0,10.0,4.333333


In [27]:
model_ids = best_models_by_group.run_id.values.tolist()
model_ids

['d091f1c8342b4c649f86931157f07617',
 'd8deb4fefd374fbfbd0fe86ab0c61fc9',
 'ecbb6e460ff3430fb9bb5411ebd09200',
 'fc43d42dd7ea45cd827823ade033d091',
 'daa9083db9664b42979f1793543e642f',
 'cdde162175474a468a8894876a96ca61',
 '31b03ec78a17495a8f00e19c03a3335b',
 'b1dfd4b6ba58496fa6c3449b59de21dc']